In [3]:
# Import libaries
import pandas as pd
import requests
import time
import re

from bs4 import BeautifulSoup

## Scrape the korean pages soup for show titles

Note: Collect Title and show URL

In [1]:
headers = {'User-agent': 'SB 1.0'}

In [ ]:
shows = []

for pg_num in range(1,36):
    KD_pg_url = 'https://www.viki.com/explore?country=korea&page=%s' %pg_num
    res = requests.get(KD_pg_url, headers=headers)
    soup = BeautifulSoup(res.content, 'lxml')

    s_page = soup.find_all('a', {'class':'thumb-title strong'})
    for item in s_page:
        show = {}
        show['title'] = item.text.strip()
        show['url'] = 'https://www.viki.com'+ item.attrs['href']
        shows.append(show)

    time.sleep(1)
k_titles = pd.DataFrame(shows)

In [ ]:
# pull out container ids
k_titles['container']= k_titles['url'].map(lambda x: re.findall('[tv]\/(.+?)\-', x)[0])

In [ ]:
# Add show rating and # ratings
k_titles['rating'] = ''
k_titles['# ratings']= ''

In [ ]:
k_titles.tail()

## Clean Data 

In [ ]:
# Drop the shows that are not tv-series
k_titles = k_titles[~k_titles.url.str.contains("movies")]
k_titles = k_titles[~k_titles.url.str.contains("creators")]
k_titles.reset_index(drop=True, inplace=True)

In [ ]:
# Find show ratings and # of ratings
for row in range(len(k_titles['container'])):
    url='https://api.viki.io/v4/containers/%s.json?app=100000a' %k_titles['container'][row]
    res = requests.get(url, headers= headers)
    json_pg = res.json()
    
    k_titles.loc[row,'rating']= json_pg['review_stats']['average_rating']
    k_titles.loc[row,'# ratings']= json_pg['review_stats']['count']
    time.sleep(1)

In [ ]:
k_titles.tail()

In [ ]:
k_titles.to_csv('./data/k_titles.csv', index=False)

## Scrape show pages for user ratings

In [4]:
k_titles = pd.read_csv('./data/k_titles.csv')

In [5]:
k_titles.head()

,title,url,container,rating,# ratings
0,Thirty But Seventeen,https://www.viki.com/tv/36109c-thirty-but-seve...,36109c,9.58,7368
1,Fates and Furies,https://www.viki.com/tv/36240c-fates-and-furies,36240c,9.14,1401
2,The Last Empress,https://www.viki.com/tv/36241c-the-last-empress,36241c,9.45,3858
3,Encounter,https://www.viki.com/tv/36239c-encounter,36239c,9.59,5436
4,My Strange Hero,https://www.viki.com/tv/36330c-my-strange-hero,36330c,9.56,2239


In [ ]:
url3='https://api.viki.io/v4/containers/36239c/reviews.json?sort=review_rank&direction=desc&per_page=10&page=1&app=100000a' 
res = requests.get(url3, headers= headers)
json_pg = res.json()
json_pg['more']

### Access to viki api

https://api.viki.io/v4/containers/36109c/reviews.json?sort=review_rank&direction=desc&per_page=20&page=3&has_review_note=1&app=100000a

*** key 'more' indicates if there are more pages boolean ***

In [13]:
ratings = []

for row in range(len(k_titles['container'])):

    pg_nums =1

    while True:
        url1='https://api.viki.io/v4/containers/%s/reviews.json?sort=review_rank&direction=desc&per_page=10&page=%s&app=100000a' %(row, pg_nums)
        res = requests.get(url1, headers= headers)
        json_pg = res.json()
        next_pg = json_pg['more']

        for row in json_pg['response']:
            rating = {}
            rating['title'] = k_titles['title'][0]
            rating['user'] = row['user']['username']
            rating['rating'] = row['user_content_rating']
            ratings.append(rating)
            
        pg_nums += 1

        time.sleep(1)

        if next_pg != True:
            break

In [15]:
show_rates = pd.DataFrame(ratings)
print(show_rates.shape[0])
print(show_rates['user'].nunique())

2287
2287


In [ ]:
k_titles.to_csv('./data/k_ratings.csv', index=False)

### Step 2: Using the show URL, scrape each show page and get user ratings

In [ ]:
k_titles = pd.read_csv('./data/k_titles.csv')

In [ ]:
show_url = k_titles['url'][5] + '#modal-review-see-all'

# use selenium webdriver as library that acts as a 
# headless browser to access modal window
driver = webdriver.Chrome(chrome_options=opts)
driver.implicitly_wait(30)
driver.get(show_url)
test = []

In [ ]:
for i in range(3428):
    time.sleep(3)
    soup_level1=BeautifulSoup(driver.page_source, 'lxml')

    table = soup_level1.find_all('ul',{'class':'media-list'})
    for row in table[2].find_all('li',{'class':'media'}):
        review = {}
        review['title'] = k_titles['title'][5]
        review['user'] = row.find('a').attrs['href'][7:]
        review['rating'] = row.find('span', {'class':'strong'}).text
        test.append(review)
            
    driver.find_element_by_partial_link_text('Next').click()


In [ ]:
review_6 = pd.DataFrame(test)
print(review_6.shape[0])
print(review_6['user'].nunique())
# reviews_5 = pd.DataFrame(reviews5)
# print(reviews_5.shape[0])
# print(reviews_5['user'].nunique())

In [ ]:
# testp

In [ ]:
%%html
<style>
table {float:left}
</style>

## scraped reviews
|movie|reported|retrieved|%|
|---|---|---|--|
|review 2 | 7106| 2195| 30.9|
|review 1 | 1354| 516| 38.1|
|review 3 | 3683| 1545| 41.9|
|review 4 | 4850| 1818| 37.5|
|review 5 | 2123| 660| 31.1|
|review 6 | 34282|||

In [ ]:
518/34282

In [ ]:
#end the Selenium browser session
driver.quit()

In [ ]:
review_6.to_csv('./data/show6_reviews.csv')

In [ ]:
reviews = []
for show in range(len(shows)):
    rest_url = k_titles['url']
    new_res = requests.get(rest_url)#modal-review-see-all
    soup = BeautifulSoup(new_res.content, 'lxml')

    user = soup.find_all('div', {'class':'media-body'})
    for item in user:
        review = {}
        review['user'] = item.find('a').attrs['href'][17:]
        show['rating'] = row.find('a').attrs['href'][12:].title()
        reviews.append(review)
# foods[100:150]

### Step 4: Create a pandas DataFrame from your list of foods

In [ ]:
df = pd.DataFrame(reviews)
df.head(10)

### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [ ]:
filepaths = ['./data/shows_part_1.csv', './data/shows_part_2.csv', './data/shows_part_3.csv','./data/shows_part_4.csv','./data/shows_part_5.csv']
df = pd.concat(map(pd.read_csv, filepaths))


In [ ]:
df

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.tail()

In [ ]:
df.to_csv('./data/pages_info.csv',index=False)